# 2 4 - Jayden

In [1]:
import pandas as pd
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from utils import read_classification_dataset



#Seed for randomness in cross-validation

train2, target2, test2 = read_classification_dataset(2)

k = 10
seed = 10
kf = KFold(n_splits=k, shuffle=True, random_state=seed)

#This was the best performing SVM after messing around with different parameters
svr = SVC(kernel='linear',gamma=0.9)

def pcareduce(data:pd.DataFrame):

    pca = PCA(n_components=10)
    pca.fit(train2)
    return pd.DataFrame(pca.transform(train2))
train2_reduced = pcareduce(train2)
test2_reduced = pcareduce(test2)


sk.metrics.get_scorer_names()
scores = cross_val_score(svr, train2_reduced, target2.values.ravel(), cv=kf, scoring='accuracy')


mean_score = scores.mean()
std_score = scores.std()
print(f"Mean Score: {mean_score:.2f}")
print(f"Standard Deviation of Scores: {std_score:.2f}")

Mean Score: 0.88
Standard Deviation of Scores: 0.06


**Start of Data 4**

KNN

In [2]:
train4, target4, test4 = read_classification_dataset(4)

train4.isna().sum().sum()


# svc4 = SVC(kernel='linear',gamma=0.9)
knnr = KNeighborsClassifier(n_neighbors=5,weights='distance')
k = 10
kf = KFold(n_splits=k,shuffle=True,random_state=42)



scores = cross_val_score(knnr,train4,target4.values.ravel(),cv=kf,scoring='accuracy')

print(f"Mean Score: {scores.mean():.2f}")
print(f"Standard Deviation of Scores: {scores.std():.2f}")


Mean Score: 0.74
Standard Deviation of Scores: 0.04


Random Forest

In [3]:
forest = RandomForestClassifier(n_estimators=50,max_depth=5)

scores = cross_val_score(forest,train4,target4.values.ravel(),cv=kf,scoring='accuracy')

print(f"Mean Score: {scores.mean():.2f}")
print(f"Standard Deviation of Scores: {scores.std():.2f}")

Mean Score: 0.87
Standard Deviation of Scores: 0.03


LASSO Regression

In [5]:
lasso = Lasso(alpha=0.5)

scores = cross_val_score(lasso,train4,target4.values.ravel(),cv=kf,scoring='r2')

print(f"Mean R Squared Score: {scores.mean():.2f}")
print(f"Standard Deviation of Scores: {scores.std():.2f}")

Mean R Squared Score: 0.57
Standard Deviation of Scores: 0.05
